# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 10** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez </br>
**Equipo**: Par de Dos

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab10_ParDeDos") \
    .master("spark://b5537c84a339:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/27 18:18:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Preparación de Datos

In [3]:
from team_ParDeDos.spark_utils import SparkUtils as SpU
# Columns: male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD

# Define schema for the DataFrame
schema = SpU.generate_schema([
        ("male", "integer"),
        ("age", "integer"),
        ("education", "integer"),
        ("currentSmoker", "integer"),
        ("cigsPerDay", "integer"),
        ("BPMeds", "integer"),
        ("prevalentStroke", "integer"),
        ("prevalentHyp", "integer"),
        ("diabetes", "integer"),
        ("totChol", "integer"),
        ("sysBP", "integer"),
        ("diaBP", "integer"),
        ("BMI", "integer"),
        ("heartRate", "integer"),
        ("glucose", "integer"),
        ("TenYearCHD", "integer")
    ])

# Create DataFrame
framingham = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/framingham.csv")

### Assemble the features into a single vector column

In [4]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ["male", "age", "education", "currentSmoker", "cigsPerDay",
                  "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes",
                  "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

data_with_features = assembler.transform(framingham).select("TenYearCDH", "features")

ModuleNotFoundError: No module named 'numpy'

### Split the data into training and test sets 80% training data and 20% testing data

In [6]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [7]:
print("Original Dataset")
data_with_features.show()

# Print train dataset
print("train set")
train_df.show()

Original Dataset


+-----+---------+
|label| features|
+-----+---------+
|  1.0|[2.0,3.0]|
|  0.0|[1.0,2.5]|
|  1.0|[3.0,5.0]|
|  0.0|[0.5,1.0]|
|  1.0|[4.0,6.0]|
+-----+---------+

train set


+-----+---------+
|label| features|
+-----+---------+
|  0.0|[1.0,2.5]|
|  1.0|[2.0,3.0]|
|  0.0|[0.5,1.0]|
|  1.0|[4.0,6.0]|
+-----+---------+



### Create a logistic regression model

In [8]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)


# TRAINNING

In [9]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

25/04/20 18:52:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/20 18:52:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Coefficients: [2.346116998875653,0.7963873036415706]


# PREDICTIONS

In [11]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction", "probability").show()

+---------+----------+--------------------+
| features|prediction|         probability|
+---------+----------+--------------------+
|[3.0,5.0]|       1.0|[0.00524886113385...|
+---------+----------+--------------------+



In [12]:
sc.stop()